# Facial Emotion Recognition 🙈🙉🙊

### Supervisor: Dr. Mahmoud Elsabagh & Eng. Hassan Saad
### El Gang Team:
- #### Mariam Mohamed Skoot
- #### Wafaa Mohamed Hegazy
- #### Rawda Mohamed Rezk
- #### Ashraqat ِAbdelnasser Effat
- #### Fatma Shehata Awis
- #### Shahd Soror Sheta
- #### Lobna Saad Salah
- #### Eslam Mohamed Elgawish
- #### Mohamed Ebrahim Hamed
- #### Mohamed Mohamed Abu Elenin
- #### Mohamed Mahmoud Khalil
-------------------

In [ ]:
!pip install tensorflow opencv-python numpy pandas kaggle

-------------------------

## Import Libraries

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator      # Image preparation and processing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


--------------
## Upload images

In [ ]:
# Data paths
train_dir = 'D:\\Myproject\\Facial Emotion-CNN\\train'
test_dir = 'D:\\Myproject\\Facial Emotion-CNN\\test'

# Normalize pixel values from [0, 255] to [0, 1] to improve model training
train_datagen = ImageDataGenerator(rescale=1./255) 
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),     # Resize images before entering the model.
    batch_size=64,            # Feed the model with images in batches
    color_mode='grayscale',   # Convert photos to black and white
    class_mode='categorical'  # One-hot handling of multiple classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

----------------
## Model


In [ ]:
model = Sequential([

    # Extract basic features from images
    Conv2D(32, (3,3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2,2),       # To reduce spatial dimensions

    # Extract more complex patterns 
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    # Extract much more complex features
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    # Convert dimensions from 2D to 1D to be suitable for dense layers
    Flatten(), 
    Dense(512, activation='relu'),
    Dropout(0.5),    # To prevent overfitting
    Dense(7, activation='softmax')       # Output layer for 7 classes
])

--------------
## Compile & Train Model

In [ ]:
# Compile
# 'adam' optimizer for efficient training
# 'categorical_crossentropy' is used as the loss function
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Train
model.fit(
    train_generator,
    epochs=50,      # Train the model for 30 epochs
    validation_data=test_generator  # Here, we use the test data instead of a separate validation set
)

------------------------
## Save Model

In [ ]:
model.save('emotion_model.h5')
print("Model saved successfully as emotion_model.h5")

--------------
## 1. Real-Time Interface using camera

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# upload train model
model = load_model('emotion_model.h5')

class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# Open camera
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_gray = gray_frame[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi = roi_gray.astype('float')/255.0
        roi = np.expand_dims(roi, axis=0)
        roi = np.expand_dims(roi, axis=-1)

        prediction = model.predict(roi)
        label = class_labels[np.argmax(prediction)]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)

    cv2.imshow('Facial Emotion Recognition', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━

------------
## Reading and processing images using cv2

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model


# Map each emotion to a suitable message
emotion_messages = {
    'Angry': "Take a deep breath. Try to relax.",
    'Disgust': "Is something bothering you?",
    'Fear': "It's okay to be afraid. You're not alone.",
    'Happy': "Keep smiling! Happiness looks great on you.",
    'Sad': "Stay strong. Better days are coming.",
    'Surprise': "Wow! That was unexpected!",
    'Neutral': "You look calm and relaxed."
}

# Load trained model
model = load_model('emotion_model.h5')

# Emotion labels
class_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

def predict_emotion(image_path):
    # Read image
    image = cv2.imread(image_path)

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Load face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in image
    faces = face_cascade.detectMultiScale(gray_image, 1.3, 5)

    if len(faces) == 0:
        return None, "No face detected"

    # Process the first detected face
    (x, y, w, h) = faces[0]
    roi_gray = gray_image[y:y+h, x:x+w]
    roi_gray = cv2.resize(roi_gray, (48, 48))
    roi = roi_gray.astype('float') / 255.0
    roi = np.expand_dims(roi, axis=0)
    roi = np.expand_dims(roi, axis=-1)

    # Predict emotion
    prediction = model.predict(roi)
    label = class_labels[np.argmax(prediction)]
    message = emotion_messages.get(label, "No message available.")

    return label, message


--------------
## Interface Using Tkinter Library

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk

# GUI
def upload_and_predict():
    # Open file dialog to choose image
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    if not file_path:
        return

    # Show image in label
    pil_img = Image.open(file_path)
    pil_img = pil_img.resize((300, 300))
    img_tk = ImageTk.PhotoImage(pil_img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

    # Predict emotion
    label, message = predict_emotion(file_path)
    if label is None:
        result_label.config(text=message)
        message_label.config(text="")
    else:
        result_label.config(text=f"Predicted Emotion: {label}")
        message_label.config(text=f"Message: {message}")

# Create main window
root = tk.Tk()
root.title("Emotion Detection")

# Upload button
upload_btn = tk.Button(root, text="Upload Image", command=upload_and_predict)
upload_btn.pack(pady=10)

# Label to show image
image_label = tk.Label(root)
image_label.pack()

# Label to show predicted emotion
result_label = tk.Label(root, text="", font=("Helvetica", 16))
result_label.pack(pady=5)

# Label to show message
message_label = tk.Label(root, text="", font=("Helvetica", 12), wraplength=300)
message_label.pack(pady=5)

root.mainloop()
